# Explo TF Prédi-Véli

Basic task: 
1. take raw data from 8:40, 8:50 and 9:00
2. predict number of free docks at 9:30.

## Load and prepare data

In [2]:
import pandas as pd
from pathlib import Path

features_source_path = Path()/"data_source/summary_features_2023-02-11.parquet"
labels_source_path = Path()/"data_source/summary_labels_2023-02-17.parquet"

In [22]:
df = pd.read_parquet(features_source_path)

In [23]:
df

,stationCode,available_mechanical,available_electrical,local_hour,local_minute,week_day,date
file_time,,,,,,,
2022-05-20 08:40:08+02:00,18024,3.0,4.0,8,40,4,2022-05-20
2022-05-20 08:40:08+02:00,19040,1.0,2.0,8,40,4,2022-05-20
2022-05-20 08:40:08+02:00,26001,11.0,8.0,8,40,4,2022-05-20
2022-05-20 08:40:08+02:00,17047,0.0,1.0,8,40,4,2022-05-20
2022-05-20 08:40:08+02:00,6016,5.0,1.0,8,40,4,2022-05-20
...,...,...,...,...,...,...,...
2023-02-11 09:00:07+01:00,11041,0.0,0.0,9,0,5,2023-02-11
2023-02-11 09:00:07+01:00,19039,0.0,5.0,9,0,5,2023-02-11
2023-02-11 09:00:07+01:00,21703,0.0,0.0,9,0,5,2023-02-11


In [31]:
df["local_minute"].unique()

array([40, 50,  0], dtype=int64)

In [4]:
len(df["stationCode"].unique())

1468

## Naïve attempt: all stations, all time

Model:
* input = all stations, mechanical and electrical, 9:00
* output = all stations, free docks, 9:30

NB: test this, check where it fails (if it fails!) and use it as benchmark for the rest. 

### Preparing features

In [10]:
input_mask = (df["local_hour"]==9) & (df["local_minute"]==0)
raw_input = df[input_mask]

In [11]:
input_mech_col = ["available_mechanical", "date", "stationCode"]

input_mech_by_date = raw_input[input_mech_col].pivot(index=["date"], columns=["stationCode"])

In [12]:
input_elec_col = ["available_electrical", "date", "stationCode"]

input_elec_by_date = raw_input[input_elec_col].pivot(index=["date"], columns=["stationCode"])

In [25]:
input_mech_by_date.shape

(235, 1468)

In [26]:
input_elec_by_date.shape

(235, 1468)

In [27]:
input_ini = input_elec_by_date.join(input_mech_by_date)

In [28]:
input_ini.columns

MultiIndex([('available_electrical', '10001'),
            ('available_electrical', '10003'),
            ('available_electrical', '10004'),
            ('available_electrical', '10005'),
            ('available_electrical', '10006'),
            ('available_electrical', '10008'),
            ('available_electrical', '10009'),
            ('available_electrical',  '1001'),
            ('available_electrical', '10010'),
            ('available_electrical', '10011'),
            ...
            ('available_mechanical',  '9117'),
            ('available_mechanical',  '9118'),
            ('available_mechanical', '92001'),
            ('available_mechanical', '92002'),
            ('available_mechanical', '92003'),
            ('available_mechanical', '92004'),
            ('available_mechanical', '92005'),
            ('available_mechanical', '92006'),
            ('available_mechanical', '92007'),
            ('available_mechanical', '92008')],
           names=[None, 'stationCode'], len

### Preparing labels

In [3]:
raw_labels = pd.read_parquet(labels_source_path)

In [4]:
raw_labels["date"] = pd.to_datetime(raw_labels["date"])

In [78]:
raw_labels["stationCode"]

file_time
2022-05-20 09:30:08+02:00     5001
2022-05-20 09:30:08+02:00    20044
2022-05-20 09:30:08+02:00    14031
2022-05-20 09:30:08+02:00     6002
2022-05-20 09:30:08+02:00    41401
                             ...  
2023-02-11 09:30:08+01:00    19038
2023-02-11 09:30:08+01:00    18109
2023-02-11 09:30:08+01:00    19009
2023-02-11 09:30:08+01:00    10020
2023-02-11 09:30:08+01:00    16107
Name: stationCode, Length: 340778, dtype: string

In [5]:
# To compute available free docks, need station capacity!

data_folder = Path()/"../../data/"

station_infos_list = sorted(list(data_folder.glob('station_info*.csv')))

station_infos_list

[WindowsPath('../../data/station_info_2022-05-15.csv'),
 WindowsPath('../../data/station_info_2022-05-16.csv'),
 WindowsPath('../../data/station_info_2022-05-17.csv'),
 WindowsPath('../../data/station_info_2022-05-18.csv'),
 WindowsPath('../../data/station_info_2022-05-19.csv'),
 WindowsPath('../../data/station_info_2022-05-20.csv'),
 WindowsPath('../../data/station_info_2022-05-21.csv'),
 WindowsPath('../../data/station_info_2022-05-22.csv'),
 WindowsPath('../../data/station_info_2022-05-23.csv'),
 WindowsPath('../../data/station_info_2022-05-24.csv'),
 WindowsPath('../../data/station_info_2022-05-25.csv'),
 WindowsPath('../../data/station_info_2022-05-26.csv'),
 WindowsPath('../../data/station_info_2022-05-27.csv'),
 WindowsPath('../../data/station_info_2022-05-28.csv'),
 WindowsPath('../../data/station_info_2022-05-29.csv'),
 WindowsPath('../../data/station_info_2022-05-30.csv'),
 WindowsPath('../../data/station_info_2022-05-31.csv'),
 WindowsPath('../../data/station_info_2022-06-01

In [83]:
def get_station_info(file_path):
    """ Get station info from path + add date from path 
    
    Arg:
        * file_path (str) in format "*/station_info_YYYY-MM-DD.csv"

    """
    names = ["stationCode", "name", "capacity", 
             "station_geo", "credit_card"]
    
    df = pd.read_csv(file_path)
    #df = pd.read_csv(file_path, names = names, index_col="stationCode")

    date = str(file_path)[-14:-4]
    df["date"] = pd.to_datetime(date)
    df["stationCode"] = df["stationCode"].astype("string")

    return df.set_index("stationCode")

In [84]:
get_station_info(station_infos_list[5])

,station_name,capacity,station_geo,credit_card,date
stationCode,,,,,
10001,René Boulanger - Lancry,62,"48.86854,2.36000",True,2022-05-20
10003,Hauteville - Bonne Nouvelle,17,"48.87077,2.34965",True,2022-05-20
10004,Metz - Faubourg Saint-Denis,48,"48.87089,2.35352",True,2022-05-20
10005,Petites Ecuries - Faubourg Poissonnière,17,"48.87397,2.34839",False,2022-05-20
10006,Petites Ecuries - Faubourg Saint-Denis,28,"48.87287,2.35422",True,2022-05-20
...,...,...,...,...,...
92004,Nanterre - Université,20,"48.90136,2.21269",True,2022-05-20
92005,Gare de Nanterre Ville,43,"48.89503,2.19579",True,2022-05-20
92006,Place Nelson Mandela,30,"48.89787,2.21828",False,2022-05-20


In [85]:
get_station_info(station_infos_list[5]).index

Index(['10001', '10003', '10004', '10005', '10006', '10008', '10009', '1001',
       '10010', '10011',
       ...
       '9117', '9118', '92001', '92002', '92003', '92004', '92005', '92006',
       '92007', '92008'],
      dtype='string', name='stationCode', length=1438)

In [86]:
def get_station_capacity():
    """ Dataframe with all capacities """

    capacity_df = pd.concat([get_station_info(path) for path in station_infos_list[5:]])

    return capacity_df

In [87]:
capacity_df = get_station_capacity()

In [88]:
capacity_df

,station_name,capacity,station_geo,credit_card,date
stationCode,,,,,
10001,René Boulanger - Lancry,62,"48.86854,2.36000",True,2022-05-20
10003,Hauteville - Bonne Nouvelle,17,"48.87077,2.34965",True,2022-05-20
10004,Metz - Faubourg Saint-Denis,48,"48.87089,2.35352",True,2022-05-20
10005,Petites Ecuries - Faubourg Poissonnière,17,"48.87397,2.34839",False,2022-05-20
10006,Petites Ecuries - Faubourg Saint-Denis,28,"48.87287,2.35422",True,2022-05-20
...,...,...,...,...,...
92004,Nanterre - Université,20,"48.90136,2.21269",True,2023-02-21
92005,Gare de Nanterre Ville,43,"48.89503,2.19579",True,2023-02-21
92006,Place Nelson Mandela,30,"48.89787,2.21828",False,2023-02-21


In [58]:
raw_labels

,stationCode,available_mechanical,available_electrical,local_hour,local_minute,week_day,date
file_time,,,,,,,
2022-05-20 09:30:08+02:00,5001,0.0,1.0,9,30,4,2022-05-20
2022-05-20 09:30:08+02:00,20044,1.0,2.0,9,30,4,2022-05-20
2022-05-20 09:30:08+02:00,14031,1.0,1.0,9,30,4,2022-05-20
2022-05-20 09:30:08+02:00,6002,1.0,0.0,9,30,4,2022-05-20
2022-05-20 09:30:08+02:00,41401,13.0,1.0,9,30,4,2022-05-20
...,...,...,...,...,...,...,...
2023-02-11 09:30:08+01:00,19038,0.0,0.0,9,30,5,2023-02-11
2023-02-11 09:30:08+01:00,18109,11.0,11.0,9,30,5,2023-02-11
2023-02-11 09:30:08+01:00,19009,3.0,5.0,9,30,5,2023-02-11


In [89]:
def get_available_docks(capacity_df, raw_labels):
    """"
    Dataframe with available docks
    """
    aux = capacity_df.reset_index().merge(raw_labels, how="outer", left_on=["stationCode", "date"], right_on=["stationCode", "date"])

    aux["capacity"] = pd.to_numeric(aux["capacity"], errors='coerce')

    aux["available_docks"] = aux["capacity"] - aux["available_mechanical"] - aux["available_electrical"]

    return aux

In [90]:
available_docks_df = get_available_docks(capacity_df, raw_labels)

In [91]:
capacity_df.index

Index(['10001', '10003', '10004', '10005', '10006', '10008', '10009', '1001',
       '10010', '10011',
       ...
       '9117', '9118', '92001', '92002', '92003', '92004', '92005', '92006',
       '92007', '92008'],
      dtype='string', name='stationCode', length=358200)

In [13]:
# Objectif maintenant : calculer le nombre de places disponibles !
# Une option : un simple join sur "stationCode" et "date" !

In [92]:
label_cols = ["available_docks", "date", "stationCode"]

label_by_date = available_docks_df[label_cols].pivot(index=["date"], columns=["stationCode"])

In [93]:
label_by_date

available_docks                                                  \
stationCode           10001 10003 10004 10005 10006 10008 10009 1001  10010   
date                                                                          
2022-05-20             62.0  14.0  43.0  11.0  23.0  13.0  18.0  13.0  29.0   
2022-05-21             48.0  15.0  43.0  12.0  25.0  14.0  15.0  13.0  11.0   
2022-05-22             54.0  16.0  42.0  13.0  20.0  12.0  17.0  14.0  27.0   
2022-05-23             58.0  13.0  43.0   6.0  12.0  18.0  17.0  10.0  32.0   
2022-05-24             54.0   8.0  33.0   2.0  22.0  17.0  17.0   6.0  29.0   
...                     ...   ...   ...   ...   ...   ...   ...   ...   ...   
2023-02-17              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2023-02-18              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2023-02-19              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2023-02-20              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2023-02-21              NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                   ...                                                        \
stationCode 10011  ... 9117  9118  92001 92002 92003 92004 92005 92006 92007   
date               ...                                                         
2022-05-20   12.0  ...  29.0  35.0  20.0  14.0   7.0  15.0  39.0  29.0  18.0   
2022-05-21    7.0  ...  29.0  35.0  20.0  14.0   7.0  17.0  37.0  27.0  18.0   
2022-05-22   16.0  ...  21.0  35.0  19.0  18.0   5.0  17.0  41.0  18.0  18.0   
2022-05-23   11.0  ...  31.0  36.0  20.0  11.0  19.0  14.0  41.0  19.0   4.0   
2022-05-24   14.0  ...  31.0  35.0  20.0  11.0  15.0  14.0  39.0  19.0   9.0   
...           ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
2023-02-17    NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2023-02-18    NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2023-02-19    NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2023-02-20    NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2023-02-21    NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                   
stationCode 92008  
date               
2022-05-20   11.0  
2022-05-21   11.0  
2022-05-22   14.0  
2022-05-23    7.0  
2022-05-24   10.0  
...           ...  
2023-02-17    NaN  
2023-02-18    NaN  
2023-02-19    NaN  
2023-02-20    NaN  
2023-02-21    NaN  

[249 rows x 1468 columns]

In [98]:
label_by_date.loc["2023-02-11"].notnull().sum()

1453

In [97]:
label_by_date.loc["2023-02-12"].notnull().sum()

0